In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.modeling import custom_business_cost_scorer
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Les modèles choisis pour l'optimisation
from lightgbm import LGBMClassifier # Supposons que LightGBM soit notre meilleur candidat
# from xgboost import XGBClassifier # Ou XGBoost

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from functools import partial
import warnings
import os
import logging

# Masquer les warnings MLflow liés à l'environnement
warnings.filterwarnings("ignore", message=".*Failed to resolve installed pip version.*")
logging.getLogger("mlflow.utils.environment").setLevel(logging.ERROR)
mlflow.set_tracking_uri("http://localhost:5001")

In [34]:
DATA_PATH = '../datas/02_preprocess/datas.csv'

try:
    df = pd.read_csv(DATA_PATH)
    print(f"Données chargées. Forme: {df.shape}")
except FileNotFoundError:
    print(f"Erreur: Le fichier {DATA_PATH} n'a pas été trouvé. Vérifier le chemin.")

# Séparation des features (X) et de la cible (y)
if 'TARGET' in df.columns:
    X = df.drop('TARGET', axis=1)
    y = df['TARGET']
    print(f"X shape: {X.shape}, y shape: {y.shape}")
else:
    print("Erreur: La colonne 'TARGET' n'a pas été trouvée. Vérifier le nom de la colonne cible.")

Données chargées. Forme: (307507, 139)
X shape: (307507, 138), y shape: (307507,)


In [35]:
# Division initiale pour avoir un ensemble de test final non touché par la CV
# Nous utiliserons X_train_full et y_train_full pour la validation croisée
X_train_full, X_test_final, y_train_full, y_test_final = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Taille de l'ensemble d'entraînement : {X_train_full.shape[0]} échantillons")
print(f"Taille de l'ensemble de test : {X_test_final.shape[0]} échantillons")
print(f"Proportion de départs dans y_train : {y_train_full.mean():.2%}")
print(f"Proportion de départs dans y_test : {y_test_final.mean():.2%}")

Taille de l'ensemble d'entraînement : 246005 échantillons
Taille de l'ensemble de test : 61502 échantillons
Proportion de départs dans y_train : 8.07%
Proportion de départs dans y_test : 8.07%


# Optimisation des hyperparamètres et du seuil de décision

Objectifs:
- Optimiser les hyperparamètres via GridSearchCV en maximisant un score aligné métier (coût négatif).
- Déterminer le seuil de décision minimisant le coût métier sur le test final.

Fonction de coût (FN > FP):
$\\text{total\\_cost} = FP \\cdot COST\\_{FP} + FN \\cdot COST\\_{FN}$

Étapes:
1. Split train/test final (test_final tenu hors CV).
2. GridSearchCV avec scorer "coût métier" et validation croisée stratifiée.
3. Logging MLflow:
   - meilleurs hyperparamètres, meilleur coût (CV)
   - modèle (pipeline) + signature + input_example
4. Balayage de seuils sur test_final pour minimiser le coût.
5. Rapport des métriques au seuil optimal + log du plot coût vs seuil.

Remarques:
- LightGBM: is_unbalance=True pour gérer le déséquilibre.
- Pour d'autres modèles: class_weight ou SMOTE.
- Conserver les figures (plots) en artefacts pour l'auditabilité.

In [36]:
experiment_name = "Credit_Scoring_Hyperparameter_Optimization"
mlflow.set_experiment(experiment_name)
print(f"MLflow Experiment set to: {experiment_name}")

2025/10/02 15:39:18 INFO mlflow.tracking.fluent: Experiment with name 'Credit_Scoring_Hyperparameter_Optimization' does not exist. Creating a new experiment.


MLflow Experiment set to: Credit_Scoring_Hyperparameter_Optimization


In [37]:
# Définis les coûts métier. Ces valeurs doivent être ajustées avec l'équipe métier.
COST_FN = 10000  # Coût élevé de la perte d'argent due à un défaut non détecté
COST_FP = 1000   # Coût de l'opportunité manquée ou de l'insatisfaction client

def business_cost_score(estimator, X, y_true, threshold=0.5, COST_FN=COST_FN, COST_FP=COST_FP):
    proba = estimator.predict_proba(X)
    if hasattr(proba, "ndim") and proba.ndim == 2:
        proba = proba[:, 1]
    y_pred = (proba >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    total_cost = (fp * COST_FP) + (fn * COST_FN)
    return -total_cost  # sklearn maximise le score

# Scorer: on minimise le coût => greater_is_better=False
business_scorer = partial(business_cost_score, threshold=0.5, COST_FN=COST_FN, COST_FP=COST_FP)

print(f"Coût Faux Négatif (FN): {COST_FN}")
print(f"Coût Faux Positif (FP): {COST_FP}")

Coût Faux Négatif (FN): 10000
Coût Faux Positif (FP): 1000


In [38]:
# Modèle candidat pour l'optimisation
model_class = LGBMClassifier
model_name = "LightGBM"

# Définition du pipeline
# LightGBM n'est pas sensible à la mise à l'échelle, donc le scaler est optionnel ici.
# Si tu avais un modèle comme LogisticRegression ou MLP, tu le mettrais.
pipeline = Pipeline([
    # ('scaler', StandardScaler()), # Décommenter si le scaler est bénéfique pour ton modèle choisi
    ('model', model_class(random_state=42, n_jobs=-1, verbosity=-1, force_row_wise=True))
])

# Paramètres à optimiser pour LightGBM
# Adapte ce dictionnaire de paramètres en fonction de tes besoins et de ton modèle.
# C'est un exemple, il faut être un peu plus agressif pour une vraie optimisation.
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__learning_rate': [0.05, 0.1],
    'model__num_leaves': [20, 31, 40],
    'model__max_depth': [-1, 7], # -1 signifie pas de limite
    'model__is_unbalance': [True], # Gestion du déséquilibre
    # 'model__scale_pos_weight': [ (y_train_full == 0).sum() / (y_train_full == 1).sum() ] # Alternative pour XGBoost
}

# Initialisation de StratifiedKFold pour la validation croisée dans GridSearchCV
cv_stratified = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print(f"\n--- Lancement de GridSearchCV pour {model_name} ---")

with mlflow.start_run(run_name=f"{model_name}_HPO_GridSearch"):
    # Log des paramètres du GridSearch
    mlflow.log_param("model_optimized", model_name)
    mlflow.log_param("scoring_metric_hpo", "negative_business_cost")
    mlflow.log_param("cv_n_splits", cv_stratified.n_splits)
    mlflow.log_dict(param_grid, "param_grid")

    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring=business_scorer,  # <-- utiliser le callable
        cv=cv_stratified,
        verbose=2,
        n_jobs=-1,
        error_score="raise"
    )

    grid_search.fit(X_train_full, y_train_full)

    best_params = grid_search.best_params_
    best_score = -grid_search.best_score_  # reconvertir en coût positif (car sklearn a inversé le signe)
    best_estimator = grid_search.best_estimator_
    print(f"\nMeilleurs hyperparamètres trouvés: {best_params}")
    print(f"Meilleur coût métier (moyenne CV): {best_score:.2f}")

    # Log des meilleurs paramètres et score dans MLflow
    mlflow.log_params({f"best_{k}": v for k, v in best_params.items()})
    mlflow.log_metric("best_cv_business_cost", best_score)
    mlflow.set_tag("grid_search_status", "completed")

    # Enregistrer le modèle optimisé (le pipeline complet)
    mlflow.sklearn.log_model(
        sk_model=best_estimator,
        name=f"{model_name}_optimized_pipeline",
        registered_model_name=f"{model_name}_CreditScoring_Optimized",
        input_example=X_train_full.iloc[:5],
        signature=infer_signature(X_train_full, best_estimator.predict_proba(X_train_full)[:, 1])
    )

    print(f"Modèle optimisé loggé dans MLflow Registry sous le nom: {model_name}_CreditScoring_Optimized")


--- Lancement de GridSearchCV pour LightGBM ---
Fitting 5 folds for each of 36 candidates, totalling 180 fits

Meilleurs hyperparamètres trouvés: {'model__is_unbalance': True, 'model__learning_rate': 0.05, 'model__max_depth': 7, 'model__n_estimators': 300, 'model__num_leaves': 31}
Meilleur coût métier (moyenne CV): 25706800.00


Successfully registered model 'LightGBM_CreditScoring_Optimized'.
2025/10/02 15:42:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LightGBM_CreditScoring_Optimized, version 1


Modèle optimisé loggé dans MLflow Registry sous le nom: LightGBM_CreditScoring_Optimized
🏃 View run LightGBM_HPO_GridSearch at: http://localhost:5000/#/experiments/791806172091042918/runs/bb8671abbdbf480182f233f79afb7dd3
🧪 View experiment at: http://localhost:5000/#/experiments/791806172091042918


Created version '1' of model 'LightGBM_CreditScoring_Optimized'.


In [39]:
print(f"\n--- Optimisation du seuil de décision sur l'ensemble de test final ---")

# Utilise le meilleur estimateur trouvé par GridSearchCV
final_model_pipeline = best_estimator

# Obtenir les probabilités sur l'ensemble de test final
y_pred_proba_test = final_model_pipeline.predict_proba(X_test_final)[:, 1]

# Générer une gamme de seuils à tester
thresholds = np.arange(0.01, 0.99, 0.01) # De 0.01 à 0.98 par pas de 0.01

cost_at_threshold = []
for t in thresholds:
    cost = -custom_business_cost_scorer(y_test_final, y_pred_proba_test, threshold=t)
    cost_at_threshold.append(cost)

# Trouver le seuil qui minimise le coût
optimal_threshold_idx = np.argmin(cost_at_threshold)
optimal_threshold = thresholds[optimal_threshold_idx]
min_business_cost = cost_at_threshold[optimal_threshold_idx]

print(f"Seuil optimal trouvé : {optimal_threshold:.2f}")
print(f"Coût métier minimum à ce seuil sur le test final : {min_business_cost:.2f}")

# Re-calculer les métriques classiques avec le seuil optimal
y_pred_optimal = (y_pred_proba_test >= optimal_threshold).astype(int)
optimal_roc_auc = roc_auc_score(y_test_final, y_pred_proba_test)
optimal_f1 = f1_score(y_test_final, y_pred_optimal)
optimal_recall = recall_score(y_test_final, y_pred_optimal)
optimal_precision = precision_score(y_test_final, y_pred_optimal)
optimal_tn, optimal_fp, optimal_fn, optimal_tp = confusion_matrix(y_test_final, y_pred_optimal).ravel()

print(f"\n--- Métriques sur le test final avec le seuil optimal ({optimal_threshold:.2f}) ---")
print(f"  ROC AUC: {optimal_roc_auc:.4f}")
print(f"  F1-Score: {optimal_f1:.4f}")
print(f"  Recall (classe 1): {optimal_recall:.4f}")
print(f"  Precision (classe 1): {optimal_precision:.4f}")
print(f"  Vrais Positifs (TP): {optimal_tp}")
print(f"  Faux Positifs (FP): {optimal_fp}")
print(f"  Faux Négatifs (FN): {optimal_fn}")
print(f"  Vrais Négatifs (TN): {optimal_tn}")

with mlflow.start_run(run_name=f"{model_name}_Threshold_Optimization_Final_Metrics", nested=True):
    mlflow.log_param("final_optimal_threshold", optimal_threshold)
    mlflow.log_metric("final_min_business_cost", min_business_cost)
    mlflow.log_metric("final_roc_auc", optimal_roc_auc)
    mlflow.log_metric("final_f1_score", optimal_f1)
    mlflow.log_metric("final_recall", optimal_recall)
    mlflow.log_metric("final_precision", optimal_precision)
    mlflow.log_metric("final_tp", optimal_tp)
    mlflow.log_metric("final_fp", optimal_fp)
    mlflow.log_metric("final_fn", optimal_fn)
    mlflow.log_metric("final_tn", optimal_tn)
    mlflow.set_tag("stage", "final_evaluation_with_optimal_threshold")

    # Plot du coût en fonction du seuil
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, cost_at_threshold, marker='o', linestyle='-', markersize=4)
    plt.axvline(x=optimal_threshold, color='r', linestyle='--', label=f'Seuil Optimal = {optimal_threshold:.2f}')
    plt.title('Coût Métier en fonction du Seuil de Classification')
    plt.xlabel('Seuil de Probabilité')
    plt.ylabel('Coût Métier Total')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    # Sauvegarder la figure comme artefact MLflow
    plot_path = "cost_vs_threshold_plot.png"
    plt.savefig(plot_path)
    mlflow.log_artifact(plot_path)
    plt.close() # Fermer la figure pour ne pas encombrer la mémoire si plusieurs plots sont générés
    print(f"Plot du coût vs seuil sauvegardé et loggé.")


--- Optimisation du seuil de décision sur l'ensemble de test final ---
Seuil optimal trouvé : 0.50
Coût métier minimum à ce seuil sur le test final : 31767000.00

--- Métriques sur le test final avec le seuil optimal (0.50) ---
  ROC AUC: 0.7671
  F1-Score: 0.2785
  Recall (classe 1): 0.6814
  Precision (classe 1): 0.1750
  Vrais Positifs (TP): 3383
  Faux Positifs (FP): 15947
  Faux Négatifs (FN): 1582
  Vrais Négatifs (TN): 40590
Plot du coût vs seuil sauvegardé et loggé.
🏃 View run LightGBM_Threshold_Optimization_Final_Metrics at: http://localhost:5000/#/experiments/791806172091042918/runs/865d9271dd3f48faa775f5394500beb0
🧪 View experiment at: http://localhost:5000/#/experiments/791806172091042918
